# to connect with spotify

In [ ]:
# …existing imports…
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from dotenv import load_dotenv
import os

load_dotenv()

CLIENT_ID     = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
REDIRECT_URI  = os.getenv("REDIRECT_URI", "http://127.0.0.1:8000/error")

if not CLIENT_ID or not CLIENT_SECRET:
    raise RuntimeError("Missing CLIENT_ID or CLIENT_SECRET in environment")

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    redirect_uri=REDIRECT_URI,
    scope="user-library-read playlist-modify-public user-top-read user-read-recently-played"
))
# …rest of your code…

NameError: name 'CLIENT_ID' is not defined

In [6]:
user = sp.current_user()
print(user['display_name'])


Ankit Sneh


In [7]:
top_tracks = sp.current_user_top_tracks(limit=10, time_range='short_term')
for track in top_tracks['items']:
    print(track['name'], "-", track['artists'][0]['name'])


Shy - Nightcore - Sinnon Nightcore
仙 - 《仙王的日常生活》主题曲 - 韦伯恩
Moves Like Jagger - Studio Recording From "The Voice" Performance - Maroon 5
death bed (coffee for your head) - Powfu
Friendships - Pascal Letoublon
The Greatest - Sia
A Sky Full of Stars - Coldplay
Paradise - Coldplay
Bluest Flame - Selena Gomez
Infinity - Jaymes Young
